In [1]:
import pandas as pd
import numpy as np
from copy import copy
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

## Carregando dataset

In [2]:
df = pd.read_csv('../datasets/breast_cancer_wisconsin.csv', index_col='id').sort_index()
df.head(5)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
8670,M,15.46,19.48,101.70,748.9,0.10920,0.12230,0.14660,0.08087,0.1931,...,19.26,26.00,124.90,1156.0,0.15460,0.2394,0.3791,0.15140,0.2837,0.08019
8913,B,12.89,13.12,81.89,515.9,0.06955,0.03729,0.02260,0.01171,0.1337,...,13.62,15.54,87.40,577.0,0.09616,0.1147,0.1186,0.05366,0.2309,0.06915
8915,B,14.96,19.10,97.03,687.3,0.08992,0.09823,0.05940,0.04819,0.1879,...,16.25,26.19,109.10,809.8,0.13130,0.3030,0.1804,0.14890,0.2962,0.08472
9047,B,12.94,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,...,13.86,23.02,89.69,580.9,0.11720,0.1958,0.1810,0.08388,0.3297,0.07834
85715,M,13.17,18.66,85.98,534.6,0.11580,0.12310,0.12260,0.07340,0.2128,...,15.67,27.95,102.80,759.4,0.17860,0.4166,0.5006,0.20880,0.3900,0.11790


## Seperando _features_ ou x

In [3]:
X = df.iloc[:,2:]
X.head(5)

,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
8670,19.48,101.70,748.9,0.10920,0.12230,0.14660,0.08087,0.1931,0.05796,0.4743,...,19.26,26.00,124.90,1156.0,0.15460,0.2394,0.3791,0.15140,0.2837,0.08019
8913,13.12,81.89,515.9,0.06955,0.03729,0.02260,0.01171,0.1337,0.05581,0.1532,...,13.62,15.54,87.40,577.0,0.09616,0.1147,0.1186,0.05366,0.2309,0.06915
8915,19.10,97.03,687.3,0.08992,0.09823,0.05940,0.04819,0.1879,0.05852,0.2877,...,16.25,26.19,109.10,809.8,0.13130,0.3030,0.1804,0.14890,0.2962,0.08472
9047,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,0.06200,0.1458,...,13.86,23.02,89.69,580.9,0.11720,0.1958,0.1810,0.08388,0.3297,0.07834
85715,18.66,85.98,534.6,0.11580,0.12310,0.12260,0.07340,0.2128,0.06777,0.2871,...,15.67,27.95,102.80,759.4,0.17860,0.4166,0.5006,0.20880,0.3900,0.11790


## Separando _labels_ ou y

In [4]:
y = df['diagnosis']
y.head(5)

id
8670     M
8913     B
8915     B
9047     B
85715    M
Name: diagnosis, dtype: object

1. Divida o conjunto de dados entre Treino (80%) e Teste (20%).

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=80/100, random_state=42, stratify=y)

In [7]:
from sklearn import linear_model
from sklearn import ensemble
from sklearn import tree
from sklearn import svm
from sklearn import neighbors
from sklearn import naive_bayes

In [8]:
modelos = {}

modelos['DT']     = tree.DecisionTreeClassifier()
modelos['SVM']    = svm.SVC()
modelos['KNN']    = neighbors.KNeighborsClassifier()
modelos['LogReg'] = linear_model.LogisticRegression(multi_class='multinomial',solver='newton-cg')
modelos['NB']     = naive_bayes.GaussianNB()
modelos['SGD']    = linear_model.SGDClassifier()
modelos['RF']     = ensemble.RandomForestClassifier()
modelos['GB']     = ensemble.GradientBoostingClassifier()

## VALIDAÇÃO CRUZADA K-FOLD

2. Aplique uma validação cruzada 5-Fold (ou seja: k-Fold, com k=5) usando diferentes algoritmos de classificação e escolha o algoritmo que obtiver melhor acurácia. Dica: usar cross_val_score do scikit-learn. Para entender melhor o 5-Fold Cross Validation.

In [8]:
scores = {}

for k,m in modelos.items():
  model = copy(m)
  scores[k] = cross_val_score(model, X_train, y_train, cv=5, n_jobs=10).mean()

scores

{'DT': 0.9076923076923077,
 'SVM': 0.9142857142857144,
 'KNN': 0.945054945054945,
 'LogReg': 0.9516483516483516,
 'NB': 0.9428571428571428,
 'SGD': 0.8351648351648352,
 'RF': 0.9626373626373625,
 'GB': 0.956043956043956}

In [9]:
best_alg = max(scores, key=scores.get)
best_alg

'RF'

3. Gere (treine) um modelo de classificação usando o melhor algoritmo obtido na questão 2 sobre todo o conjunto de treino.

In [11]:
model = modelos[best_alg]
model.fit(X_train, y_train)

RandomForestClassifier()

4. Calcule a acurácia do modelo gerado na questão 3 sobre o conjunto de teste.

In [12]:
model.score(X_test, y_test)

0.9736842105263158

## Escalonamento de _features_

5. Reexecute as questões de 1 a 4 só que usando os seguintes escalonamentos de características (features): MinMaxScaler e StandardScaler. Dica: use cada um separadamente para obter uma acurácia final para cada um deles.

In [27]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### MinMaxScaler

In [28]:
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns, index=X_test.index)

for k,m in modelos.items():
  model = copy(m)
  scores[k] = cross_val_score(model, X_train, y_train, n_jobs=10).mean()

best_alg = max(scores, key=scores.get)

model = copy(modelos[best_alg])
model.fit(X_train_scaled, y_train)
print(best_alg, model.score(X_test_scaled, y_test))

RF 0.9035087719298246


### StandartScaler

In [29]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns, index=X_test.index)

for k,m in modelos.items():
  model = copy(m)
  scores[k] = cross_val_score(model, X_train, y_train, n_jobs=10).mean()

best_alg = max(scores, key=scores.get)

model = copy(modelos[best_alg])
model.fit(X_train_scaled, y_train)
print(best_alg, model.score(X_test_scaled, y_test))

RF 0.9649122807017544


## PIPELINE

6. Refaça a questão 5, só que usando Pipeline.

In [34]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
  ('transform', MinMaxScaler()),
  ('estimator', copy(modelos['RF'])),
])

pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9736842105263158

In [35]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
  ('transform', StandardScaler()),
  ('estimator', copy(modelos['RF'])),
])

pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9736842105263158

## GRID SEARCH COM VALIDAÇÃO CRUZADA

### 7. Use um Grid Search com Validação Cruzada 5-Fold sobre o conjunto de treino e:

a) obtenha os melhores hiper-parâmetros do Gradient Boosting para os seguintes hiper-parâmetros:

learning_rate: 0.1, 0.05, 0.01, 0.005, 0.001  
n_estimators: 10, 50, 100, 200, 400  
max_depth: 3, 5, 7, 9  
subsample: 0.5, 0.8, 1

In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
param_grid = [{
  'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001],
  'n_estimators': [10, 50, 100, 200, 400],
  'max_depth': [3, 5, 7, 9],
  'subsample': [0.5, 0.8, 1.0]
},]

gb = copy(modelos['GB'])

grid_search = GridSearchCV(
  gb,
  param_grid,
  cv=5,
  scoring='accuracy',
  n_jobs=10
)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=10,
             param_grid=[{'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001],
                          'max_depth': [3, 5, 7, 9],
                          'n_estimators': [10, 50, 100, 200, 400],
                          'subsample': [0.5, 0.8, 1.0]}])

In [14]:
print(
grid_search.best_estimator_,
grid_search.best_params_,
grid_search.best_score_,
sep='\n'
)

GradientBoostingClassifier(n_estimators=400, subsample=0.5)
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 400, 'subsample': 0.5}
0.9758241758241757


b) Gere (treine) um modelo de classificação usando os melhores hiper-parâmetros obtidos no item (a) sobre todo o conjunto de treino.

In [23]:
from sklearn.metrics import accuracy_score

gb = ensemble.GradientBoostingClassifier(n_estimators=400, subsample=0.5, learning_rate=0.1, max_depth=3)
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)


c) Calcule a acurácia do modelo gerado no item (b) sobre o conjunto de teste.

In [24]:
accuracy_score(y_test, y_pred)

0.9649122807017544

## RANDOMIZED SEARCH COM VALIDAÇÃO CRUZADA

8. Refaça a Questão 7, só que usando o Randomized Search com Validação Cruzada ao invés do Grid Search com Validação Cruzada.

In [12]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = [{
  'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001],
  'n_estimators': [10, 50, 100, 200, 400],
  'max_depth': [3, 5, 7, 9],
  'subsample': [0.5, 0.8, 1.0]
},]

gb = copy(modelos['GB'])

rd_search = RandomizedSearchCV(
  gb,
  param_grid,
  cv=5,
  scoring='accuracy',
  n_jobs=10
)

rd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=10,
                   param_distributions=[{'learning_rate': [0.1, 0.05, 0.01,
                                                           0.005, 0.001],
                                         'max_depth': [3, 5, 7, 9],
                                         'n_estimators': [10, 50, 100, 200,
                                                          400],
                                         'subsample': [0.5, 0.8, 1.0]}],
                   scoring='accuracy')

In [13]:
print(
rd_search.best_estimator_,
rd_search.best_params_,
rd_search.best_score_,
sep='\n'
)

GradientBoostingClassifier(learning_rate=0.05, max_depth=7, n_estimators=200,
                           subsample=0.5)
{'subsample': 0.5, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05}
0.9648351648351647


In [14]:
gb = ensemble.GradientBoostingClassifier(n_estimators=200, subsample=0.5, learning_rate=0.05, max_depth=7)
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)

In [16]:
accuracy_score(y_test, y_pred)

0.956140350877193